# 1. 语义分割(Semantic Segmentation)

**语义分割** 通过对图像所有像素进行密集的推断来实现细粒度的预测，从而使每个像素都被标记为其封闭对象区域的类别。目前广泛应用于医学影像与无人驾驶等。

下图分别为图像分类、目标检测、语义分割和实例分割的示例。

![img](images/chapter16/object_recongnition.png)

### 语义分割评价指标

- 像素精度（pixel accuracy），也是最简单的度量，为标记正确的像素占总像素的比例
$$ \text{pixel accuracy} = \frac{\sum_i n_{ii}}{\sum_i{t_i}} $$


- 平均精度（mean accuracy），为所有类被正确分类像素数比例的均值
$$ \text{mean accuracy} = \frac{1}{C}\sum_i \frac{n_{ii}}{t_i} $$


- 平均交并比（mean Intersection over Union，mean IU），计算预测像素和真实像素两个集合的交集和并集之比
$$\text{mean IU} = \frac{1}{C}\sum_i \frac{n_{ii}}{t_i+\sum_j n_{ji}-n_{ii}} $$


- 频权 IU（Frequency Weighted IU，f.w.IU）
$$\text{f.w.IU} = {(\sum_k t_k)}^{-1}\sum_i \frac{t_in_{ii}}{t_i+\sum_j n_{ji}-n_{ii}}$$

其中，$n_{ji}$ 代表第 $j$ 类被识别为第 $i$ 类的像素个数，$t_i$ 代表标签中第 $i$ 类的像素个数，$C$ 代表类别数。

### 常用的语义分割数据集

- [PASCAL VOC](http://host.robots.ox.ac.uk/pascal/VOC/)

      VOC 数据集不仅包含了目标检测任务，也包含了语义分割任务，数据集包含 20 类目标和 1 类背景。VOC 还提供了了算法精度排名

[Leaderboard](http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=6)

- [MS COCO (Common Objects in Context)](http://cocodataset.org/)

![img](https://cocodataset.org/images/coco-examples.jpg)

- [Cityscapes](https://www.cityscapes-dataset.com/)

![img](images/chapter16/Cityscapes.jpg)
       Cityscapes 数据集是 2016 年推出的用于自动驾驶环境下的图像分割数据集，可以评估视觉算法在城区场景语义理解方面的性能。Cityscapes 包含50个欧洲城市不同场景、不同背景、不同季节的街景的33类标注物体，图像分辨率较大（1024×2048）
       
       
- [ADE20K_MIT](http://groups.csail.mit.edu/vision/datasets/ADE20K/)

![img](images/chapter16/ADE20K.jpg)
      ADE20K 是一个场景理解的数据集，它包含 151 类目标和 1 类背景，包括各种物体（人、汽车等）、场景（天空、路面等）。训练集包含 20,210 幅图像，验证集包含 2,000 幅图像，测试集暂未公开。

### 经典语义分割网络模型

- [**FCN**（2014）](https://arxiv.org/abs/1411.4038)
- [DeepLabv1（2014）](https://arxiv.org/abs/1412.7062)
- [CRF-RNN（2015）](https://arxiv.org/abs/1502.03240)
- [**U-Net**（2015）](https://arxiv.org/abs/1505.04597)
- [SegNet（2015）](https://arxiv.org/abs/1511.00561)
- [DeepLabv2（2016）](https://arxiv.org/abs/1606.00915)
- [FC-DenseNet（2016）](https://arxiv.org/abs/1611.09326)
- [PSPNet（2016）](https://arxiv.org/abs/1612.01105)
- [RefineNet（2017）](http://openaccess.thecvf.com/content_cvpr_2017/papers/Lin_RefineNet_Multi-Path_Refinement_CVPR_2017_paper.pdf)
- [TuSimple（2017）](https://arxiv.org/abs/1702.08502)
- [DeepLabv3（2017）](https://arxiv.org/abs/1706.05587)
- [DeepLabv3+（2018）](https://arxiv.org/abs/1802.02611)
- [nnU-Net(2019)](https://arxiv.org/abs/1904.08128)

# 2. [FCN](https://arxiv.org/abs/1411.4038)

经典的 CNN 模型在卷积层之后使用全连接层获得固定长度的特征向量进行分类，由于全连接层必须是固定尺寸输入，导致整个网络的输入尺寸无法改变。FCN（Fully Convolutional Network）的作者 Jonathan Long 意识到了这一限制来自于网络最后的全连接层，于是将最后的全连接层改为 1×1 卷积层实现。

下图中的猫, 输入经典 AlexNet 网络中，输出为长度 1000 的特征向量，向量中每个元素代表输入图像属于每一类的概率。其中在对应“tabby cat”这一类上数值最高，因此判定输入图像属于“tabby cat”类。

如果将分类网络改成全卷积网络，则网络输出了一个低分辨率的类的热图（heatmap）。
![img](images/chapter16/transform_fc_to_conv.png)

如果使用**反卷积层**（Deconvolution Layer）对最后一个卷积层的特征图进行上采样，使它恢复到与输入图像相同的尺寸，从而可以对每个像素都产生一个分类输出。

![img](images/chapter16/FCN.png)

## 反卷积

反卷积（Deconvolution）这个概念容易引起误会，转置卷积（Transposed Convolution）是一个更为合适的叫法。

例如，对于 4×4 的输入图像，使用 3×3 的卷积核对其进行卷积，步长为 1 ，没有填充（即$i=4, k=3, s=1, p=0$）。因此输出的图像尺寸为 2×2，如下图所示。

![img](images/chapter16/no_padding_no_strides.gif)

将输入图像展开为 16 维向量 $x$，将输出图像展开为 4 维向量 $y$，则卷积运算表示为 $y = Cx$，其中 $C$ 可以写为如下的稀疏矩阵

$$ 
\begin{equation}
\left(
  \begin{array}{ccccc}
    w_{0,0} & w_{0,1} & w_{0,2} & 0 & w_{1,0} & w_{1,1} & w_{1,2} & 0 & w_{2,0} & w_{2,1} & w_{2,2} & 0 & 0 & 0 & 0 & 0\\
    0 & w_{0,0} & w_{0,1} & w_{0,2} & 0 & w_{1,0} & w_{1,1} & w_{1,2} & 0 & w_{2,0} & w_{2,1} & w_{2,2} & 0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0 & w_{0,0} & w_{0,1} & w_{0,2} & 0 & w_{1,0} & w_{1,1} & w_{1,2} & 0 & w_{2,0} & w_{2,1} & w_{2,2} & 0\\
    0 & 0 & 0 & 0 & 0 & w_{0,0} & w_{0,1} & w_{0,2} & 0 & w_{1,0} & w_{1,1} & w_{1,2} & 0 & w_{2,0} & w_{2,1} & w_{2,2}\\
  \end{array}
\right)
\end{equation}
 $$
 
转置卷积运算则是在 2×2 图像上乘以 $C^T$，可以记作 $y'=C^Tx'$

另一个理解反卷积运算的方式是，使用一个低效的卷积运算来实现：在 2×2 图像应用  3×3 的卷积核，并且填充幅度为 2（$i'=2,k'=k,s'=1,p'=2$），如下图所示

![img](images/chapter16/no_padding_no_strides_transposed.gif)

## 跳跃结构

FCN 模型进一步采用了**跳跃结构**（skip architecture），使用了**融合**（fusion）策略将高层特征和低层特征进行相加来提高模型分割性能。网络分别采用了 3 种不同的结构，分别是 FCN-32s（32 倍上采样）、FCN-16s（16 倍上采样）和 FCN-8s（8 倍上采样）。 

- FCN-32s
      输入图像经过 CNN 不断的卷积和池化，得到 pool5 层输出的特征图（特征图宽高是原图的 1/32），将该特征图通过 32 倍的上采样（反卷积层）得到最后输出结果。

 
- FCN-16s
      将 pool5 的输出特征图进行 2 倍上采样，与 pool4 经过 1×1 卷积层后输出的特征图进行融合，然后将融合结果进行 16 倍的上采样得到最后输出结果。 

- FCN-8s
      将 pool5 和 pool4 的融合结果进行 2 倍的上采样，再与 pool3 经过 1×1 卷积层后输出的特征图进行融合，之后进行 8 倍的上采样得到最后输出结果。
      
![img](images/chapter16/FCN_combine_coarse_with_fine.png)

三种网络结构实验结果表明，将高层特征与低层特征的融合能够明显提高像素点的分类精度
![img](images/chapter16/FCN_result.png)

|模型|Pixel Accuracy|Mean Pixel Accuracy|Mean IU|f.w.IU|
|:----|:---:|:---:|:---:|:---:|
|FCN-32s-fixed | 83.0| 59.7| 45.4| 72.0|
|FCN-32s| 89.1| 73.3| 59.4| 81.4|
|FCN-16s |90.0| 75.7| 62.4 | 83.0|
|FCN-8s| **90.3** | **75.9** |**62.7** |**83.2**|

FCN 模型影响比较深远，不仅是深度学习模型在语义分割领域的首次成功应用。而且自从 FCN 提出后，一些经典的目标检测和识别模型都采用了 FCN 的概念，例如 Faster R-CNN 模型和 SSD 模型。 

# 3. [带孔卷积](https://arxiv.org/abs/1511.07122)

在深度卷积神经网络中重复最大池化下采样会带来分辨率下降的问题，低分辨率的特征图会导致细节丢失。![img](images/chapter16/without_atrous_convolution.png)

|输出图像缩放率(output stride)|8|16|32|64|128|256|
|---|---|----|--|--|--|--|
|mean IU|75.18|73.88|70.06|59.99|42.34|20.29|

为了缓解这个问题，学者将带孔卷积引入到卷积神经网络中用于图像分割任务。带孔卷积、空洞卷积或膨胀卷积（Dilated Convolution 或 Atrous Convolution）是在标准的卷积核里加入空洞，以此来增加卷积层的感受野。带孔卷积的好处是不做池化损失信息的情况下，加大了感受野，让每个卷积输出都包含较大范围的信息。

相比标准的卷积运算，带孔卷积多了一个超参数，称之为带孔率（dilation rate），记作 $d$，指的是卷积核内的间隔数量。标准的卷积运算中 $d=1$，带孔卷积中 $d>1$。

带孔卷积卷积核的有效尺寸 $\hat k$ 为

$$\hat k= k+(k-1)(d-1)$$

输出图像尺寸记作 $o$，则有

$$ o = \lfloor \frac{i + 2p -k-(k-1)(d-1)}{s} \rfloor + 1 $$

![img](images/chapter16/atrous_convolution.png)

下图中的示例中，$i=7,k=3,d=2,s=1,p=0$，因此 $o=3$

![img](images/chapter16/dilation_conv.gif)

如果多次叠加带孔率相同的卷积层的话，则会出现 **Gridding** 问题，如下图(a)，叠加了三层 $d=2$ 的 $3\times3$ 卷积层后，输入层中 $13\times 13$ 大小的区域内仅有 $7\times7$ 的像素对输出层有共享作用。可以通过叠加 $d=1$、$d=2$ 和 $d=3$ 解决这个问题，如下图(b)所示。

![img](images/chapter16/gridding_problem.png)

# 4. DeepLab [v1](https://arxiv.org/abs/1412.7062)，[v2](https://arxiv.org/abs/1606.00915)，[v3](https://arxiv.org/abs/1706.05587)，[v3+](https://arxiv.org/abs/1802.02611)

DeepLab 系列网络模型采用带孔卷积算法扩展感受野，获取更多的上下文信息。

## 4.1 [DeepLab v1](https://arxiv.org/abs/1412.7062)

- 主干网采用 VGG 16 网络，其中最后两个池化层降采样被带孔率为2的带孔卷积取代

- 使用 ImageNet 数据集预训练网络参数，再在 PASCAL VOC 数据集上微调(finetune)

- 采用完全连接的条件随机场(Fully Connected Conditional Random Fields)提高模型捕获细节的能力

![img](images/chapter16/DeepLab_v1_2.png)

## 4.2 [DeepLab v2](https://arxiv.org/abs/1606.00915)

- 使用 ASPP(Atrous Spatial Pyramid Pooling) 模块利用多尺度信息获得更好的分割效果
- 主干网由 VGG16 换为 ResNet-101
- 使用不同的学习率策略(learning rate policy)

$$ \eta_{iter} = (1 - \frac{iter}{max\_iter})^{power}\eta_0 $$

![img](images/chapter16/ASPP.png)
![img](images/chapter16/DeepLab-ASPP.png)

## 4.3 [DeepLab v3](https://arxiv.org/abs/1706.05587)

- 提出了更通用的框架，适用于任何网络
- 复制了 ResNet 中的 Block4，并级联起来
- 改进了 ASPP 模块，并在 ASPP 中使用 BN 层
- 不再使用条件随机场

当我们使用不同带孔率的 $3\times3$ 卷积核应用在 $65 \times 65$ 的特征图上时，当卷积核的带孔率接近特征图大小时，$3\times3$ 的卷积核不能捕捉全图像的上下文，而是退化为简单的 $1\times1$ 卷积核，此时只有卷积核中心点的权重起了作用。

![img](images/chapter16/valid_weights_in_atrous_conv.png)

针对上面的问题，DeepLab v3 改进了 ASPP 模块：

(a) 改进后的 ASPP 模块包括一个 $1\times1$ 卷积和三个 $3\times3$ 卷积，其中 $3\times3$ 卷积的带孔率分别为 6、12 和 18。

(b)  ASPP 采用全局平均池化(global average pooling)对模型特征图进行池化，再将得到的图像级特征输入到 $1\times1$ 卷积中，然后将特征进行双线性上采样到特定的空间维度。

![img](images/chapter16/DeepLab_v3_augmented_ASPP.png)

## 4.4 [DeepLab v3+](https://arxiv.org/abs/1802.02611)

DeepLab v3+ 采用[深度可分离卷积（Depthwise separable convolution）](https://arxiv.org/abs/1704.04861)，在降低计算量的同时可以保持较高的分割精度。

![img](images/chapter16/depthwise_separable_convolution.png)

网络整体结构如图所示：

![img](images/chapter16/DeepLab_v3+.png)

- 采用编码解码（Encoder-Decoder）结构
- Encoder 结构与 DeepLab v3 基本相同，输入与输出尺寸比变为16（output stride = 16）
- Decoder中，先将 Encoder 输出特征图上采样 4 倍，然后与 DCNN 中 0.25x 的特征图拼接一起，再进行 $3\times3 $ 的卷积，最后上采样 4 倍得到最终结果

# 5. [U-Net](https://arxiv.org/abs/1505.04597)

U-Net 最早发表在 2015 的 [MICCAI](http://www.miccai.org/) 国际会议上，目前论文引用数超过 17000+。它采用了典型的对称编码解码结构，网络结构如下图所示，因为形似英文字母 U 所以被称为 U-Net。

网络首先反复进行卷积和池化操作进行下采样，然后通过反卷积进行上采样，同时与此前的特征图进行通道融合。

![img](images/chapter16/U-Net.png)

图中蓝/白色框表示特征图，蓝色箭头表示 $3\times3$ 卷积，用于特征提取；灰色箭头表示 skip-connection，用于特征融合；红色箭头表示 $2\times2$ 最大池化；绿色箭头表示 2 倍上采样；青色箭头表示 $1\times1$ 卷积，用于输出结果。

与 FCN 不同的是，FCN 中深层信息与浅层信息融合是通过对应像素相加的方式，而 U-Net 是通过通道拼接的方式。与 Deeplab v3+ 等自然图像语义分割网络模型相比，U-Net 网络参数量规模少很多。

医学影像数据（如 CT、MRI）多是三维数据，即数据由多张二维切片组成。针对三维影像数据的分割问题，避免将逐张切片送入模型进行训练和推理，U-Net 作者提出了 [3D U-Net](https://arxiv.org/abs/1606.06650) 网络模型，其网络结构与 2D U-Net 基本相似，只是卷积核尺寸变为 $3\times 3\times 3$，最大池化变为 $2\times 2\times 2$。

![img](images/chapter16/3D_U-Net.png)

# 6. [nnU-Net](https://arxiv.org/abs/1904.08128)

Fabian Isensee 等人提出一种鲁棒的基于 2D U-Net 和 3D U-Net 的自适应框架 **nnU-Net**(no new U-Net)。

nnU-Net 在19个国际竞赛中取得最优结果，而且在49个分割任务的大多数中，都达到了 SOTA 级别，且该方法不需要人工进行调参。结果证明了这种深度学习的自适应机制具有巨大的潜力。

![img](images/chapter16/nnU-Net.png)

在 nnU-Net 中，超参数被分为蓝图参数(Blueprint Parameters)、推理参数(Inferred Parameters)和经验参数(Empirical Parameters)。

#### 蓝图参数
- 基础的网络架构选择，例如朴素的 U-Net 网络，而没有使用任何残差连接、致密连接和注意力机制
- 易于选择的一些表现较好的常用超参数：比如损失函数、优化算法、数据增强方式
- 使用 [Leaky ReLU](http://ai.stanford.edu/~amaas/papers/relu_hybrid_icml2013_final.pdf) 代替 ReLU，使用 [Instance Normalization](https://arxiv.org/abs/1607.08022) 代替 Batch Normalization


#### 推理参数
- 对一个新数据集进行适应性的编码，包括图像灰度归一化、重采样
- 根据GPU显存自动设置batch_size、patch_size和池化操作次数等

#### 经验参数
- 每次训练，nnU-Net 都会创建三个不同的网络配置：2D U-Net、3D U-Net、3D U-Net Cascade
- 进行交叉验证，nnU-Net 会经验性的选择表现最好的参数，可能是独个的推理结果，也可能是一起推理的结果

nnU-Net 验证了：

- 超参数的选择对结果的影响要比网络结构变化对结果的影响大

- nnU-Net 可以自动适应任何新数据集，很好的掌握了目标结构和图像属性

- nnU-Net 在多个不同任务中的表现都比专门设计的结构要好